# Manual flow cytometry analysis with Python
## Table of contents
1. [Why analyse flow cytometry data in Python?](#why-python)
    


## Why analyse flow cytometry data in Python?<a name="why-python"></a>
With easy to use, graphical tools like FlowJo, FCSExpress and Cytobank, you might wonder why someone would want to manually analyse their flow cytometry data by writing Python code. It’s true that for most basic cytometry experiments it will be easier and faster to use one of these platforms, but there are benefits to using Python:

- It’s free and can be run on any computer without a license!
- You can create re-useable analysis scripts that allow others to reproduce your analyses
- Analysis scripts can be automatically run periodically to analyse new screening data in a folder
- Access to a broader range of computational, statistical and plotting tools
- Managing plugins (packages) is usually easier!
- It makes you look cool ;)

In this tutorial we're going to use the `flowkit` package as it's probably the most mature and fully featured, but others exist such as `CytoPy` and `pytometry`. The `flowkit` package uses the `bokeh` package for its interactive graphics, so we import the `show` method from its `plotting` module. So that these interactive plots render in this notebook, we also need to call `bokeh.io.output_notebook()`. Don't forget that before using these packages you will need to install them, for example using `pip install bokeh flowkit`.

In [3]:
import bokeh
from bokeh.plotting import show
import flowkit as fk
bokeh.io.output_notebook()

Loading BokehJS ...

## Working with single .fcs files: the *sample* class
We’ve already seen data structures such as lists, dictionaries, NumPy arrays, and pandas DataFrames. The flowkit package provides a new data structure to contain data from an .fcs file: the *sample*. 

To read an .fcs file into Python, we use `Sample()` function from flowkit. If we call the sample object we just created, we get a print out just telling us the version of the .fcs file (version 3.0 in this case), the number of parameters (or "channels" but this technically has a different meaning in flow and we should avoid calling parameters "channels". Anyway, there are 10 here), and the number of events.

In [16]:
sample = fk.Sample('data/raw/OpA_I2_C2_IM1_WB1_R1.fcs')
sample

Sample(v3.0,                          , 10 channels, 400000 events)

In [17]:
sample.channels


,channel_number,pnn,pns,png,pne,pnr
0,1,FSC-A,,1.00,"(0.0, 0.0)",262144.0
1,2,FSC-H,,1.00,"(0.0, 0.0)",262144.0
2,3,SSC-A,,1.00,"(0.0, 0.0)",262144.0
3,4,FITC-A,CD4,1.00,"(0.0, 0.0)",262144.0
4,5,PE-A,CCR7,1.00,"(0.0, 0.0)",262144.0
5,6,PerCP-Cy5-5-A,CD8,1.00,"(0.0, 0.0)",262144.0
6,7,PE-Cy7-A,CD3,1.00,"(0.0, 0.0)",262144.0
7,8,APC-A,CD45RA,1.00,"(0.0, 0.0)",262144.0
8,9,APC-Cy7-A,CD45,1.00,"(0.0, 0.0)",262144.0
9,10,Time,,0.01,"(0.0, 0.0)",262144.0


In [19]:
sample.pnn_labels

['FSC-A',
 'FSC-H',
 'SSC-A',
 'FITC-A',
 'PE-A',
 'PerCP-Cy5-5-A',
 'PE-Cy7-A',
 'APC-A',
 'APC-Cy7-A',
 'Time']

In [22]:
sample.get_events(source = 'raw')

array([[1.51025281e+05, 1.16233000e+05, 1.92647828e+05, ...,
        7.40429993e+02, 6.16455029e+03, 1.00000001e-03],
       [1.16219039e+05, 9.49560000e+04, 6.08364609e+04, ...,
        2.73942017e+03, 3.36955508e+04, 2.00000003e-03],
       [1.45838562e+05, 1.11304000e+05, 1.97575422e+05, ...,
        2.71890015e+02, 2.48805005e+03, 3.00000012e-03],
       ...,
       [1.48042719e+05, 1.07230000e+05, 1.53985562e+05, ...,
        5.19840027e+02, 1.42956006e+03, 2.00010996e+02],
       [1.40678719e+05, 1.03240000e+05, 2.42794891e+05, ...,
        9.09720032e+02, 1.33209004e+04, 2.00010996e+02],
       [2.01164328e+05, 1.45805000e+05, 2.46083188e+05, ...,
        4.25790009e+02, 3.51405005e+03, 2.00010996e+02]])

In [25]:
sample.as_dataframe(source = 'raw')

pnn,FSC-A,FSC-H,SSC-A,FITC-A,PE-A,PerCP-Cy5-5-A,PE-Cy7-A,APC-A,APC-Cy7-A,Time
pns,,,,CD4,CCR7,CD8,CD3,CD45RA,CD45,
0,151025.281250,116233.0,192647.828125,351.140015,873.000000,172.660004,964.180054,740.429993,6164.550293,0.001000
1,116219.039062,94956.0,60836.460938,8130.540039,2537.520020,535.440002,28739.160156,2739.420166,33695.550781,0.002000
2,145838.562500,111304.0,197575.421875,353.080017,1519.020020,89.240005,863.300049,271.890015,2488.050049,0.003000
3,145582.078125,111470.0,203096.671875,492.760010,5422.300293,356.960022,2227.120117,579.690002,2491.469971,0.003000
4,85385.437500,69691.0,46193.339844,5377.680176,3166.080078,816.740051,43981.742188,2547.900146,25853.490234,0.005000
...,...,...,...,...,...,...,...,...,...,...
399995,140499.515625,112582.0,221049.421875,100.880005,1140.720093,194.000000,1994.320068,478.800018,8818.469727,200.009004
399996,158901.125000,121242.0,128474.562500,5032.360352,1689.739990,31.040001,1313.380005,1451.790039,14246.010742,200.010000
399997,148042.718750,107230.0,153985.562500,413.220001,995.220032,182.360001,234.740005,519.840027,1429.560059,200.010996


In [8]:
sample.get_metadata()

{'beginanalysis': '0',
 'endanalysis': '0',
 'beginstext': '0',
 'endstext': '0',
 'begindata': '2677',
 'enddata': '16002676           ',
 'fil': '                         ',
 'sys': 'Windows XP 5.1',
 'tot': '400000             ',
 'par': '10',
 'mode': 'L',
 'byteord': '4,3,2,1',
 'datatype': 'F',
 'nextdata': '0',
 'creator': 'BD FACSDiva Software Version 6.1.3',
 'tube name': '         ',
 'src': '           ',
 'experiment name': '                       ',
 'guid': '244147d5-4360-4cbf-9e13-fc38ab494ba4',
 'date': '19-APR-2018',
 'btim': '13:45:54',
 'etim': '13:48:37',
 'cyt': 'FACSCanto',
 'cytnum': '1',
 'window extension': '7.00',
 'export user name': '             ',
 'export time': '14-MAY-2018-13:48:34',
 'op': '             ',
 'fsc asf': '1.12',
 'autobs': 'TRUE',
 'inst': ' ',
 'laser1name': 'Blue',
 'laser1delay': '0.00',
 'laser1asf': '1.94',
 'laser2name': 'Red',
 'laser2delay': '28.48',
 'laser2asf': '1.71',
 'timestep': '0.01',
 'spill': '6,FITC-A,PE-A,PerCP-Cy5-5-A

In [32]:
sample.get_metadata()['cyt']

'FACSCanto'